# Chapter 4 : Unicode Text Versus Bytes


## Basic concepts


### what is str, character, code-point, encoding, bytes


Humans use text  
Computers speak bytes


| terms                          | definition                                                                                                                                                                  | example                                                   |
| ------------------------------ | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------- |
| code-point                     | 1 `Character` = 1 `Code-point` <br/> **purpose**: every character &rarr; number-id <br/> Fonts work on character to show [Grapheme](https://en.wikipedia.org/wiki/Grapheme) | assing A to a number(code)                                |
| code-point in Unicode standard | U+0000 to U+10FFFF (ie 1,114,111 code-points !)                                                                                                                             | A &rarr; U+0041                                           |
| encoding                       | algorithm that converts **code-points** to **byte-sequences** and vice versa <br/> **purpose**: write it down in memory or disk                                             | ascii, UTF-8, UTF-16LE, latin_1, cp1252                   |
| UTF-8                          | an encoding algorithm                                                                                                                                                       | A(U+0041) &rarr; \x41 <br/> €(U+20AC) &rarr; \xe2\x82\xac |
| UTF-16LE                       | an encoding algorithm                                                                                                                                                       | A(U+0041) &rarr; \x41\x00 <br/> €(U+20AC) &rarr;\xac\x20  |


![example of encoding: address in book: Basic Encoders/Decoders | page : 123](./images/example_encoding.jpg)


| purpose                                                              | change                                     | dechange                                   | example : Identity function                                                                                                    |
| -------------------------------------------------------------------- | ------------------------------------------ | ------------------------------------------ | ------------------------------------------------------------------------------------------------------------------------------ |
| **character <span style="font-size: larger;"> ⇔ </span> code-point** | ord(character) -> str                      | chr(code-point:int) -> int                 | chr(ord('😸'))<br/> ord(chr(0x1f638))                                                                                          |
| **character <span style="font-size: larger;"> ⇔ </span> bytes**      | my_str.encode(encoding="utf8")             | my_bytes.decode(encoding="utf8")           | '😸'.encode(encoding="utf8").decode(encoding="utf8") <br/> b'\xf0\x9f\x98\xb8'.decode(encoding="utf8").encode(encoding="utf8") |
| **int <span style="font-size: larger;"> ⇔ </span> bytes**            | my_int.to_bytes(length=1, byteorder="big") | int.from_bytes(my_binary, byteorder="big") | my_int = 65 <br/> my_binary = my_int.to_bytes(length=1, byteorder="big") <br/> int.from_bytes(my_binary, byteorder="big")      |


In [6]:
ord("A"), chr(65), ord("😸"), chr(128568)

(65, 'A', 128568, '😸')

In [10]:
my_int = 4660
my_binary = my_int.to_bytes(length=2, byteorder="big")
print(my_binary)
wrong_int = int.from_bytes(my_binary, byteorder="little")
correct_int = int.from_bytes(my_binary, byteorder="big")
wrong_int, correct_int

b'\x124'


(13330, 4660)

**Note:**

> **'cafe\u0301'**, **'caf\N{Latin Small Letter E with Acute}'**, **caf\u00E9** **'café'** are the same !  
> **0x41** , **65** both are int type

```python
>>> for word in 'cafe\u0301', 'caf\N{Latin Small Letter E with Acute}', 'caf\u00E9', 'café':
>>>     print(word)
café, café, café, café
```


In [30]:
for word in "cafe\u0301", "caf\N{LATIN SMALL LETTER E WITH ACUTE}", "caf\u00e9", "café":
    print(word, end=", ", flush=True)

café, café, café, café, 

In [31]:
word = "café"
encoded_word = word.encode("utf8")
decoded_word = encoded_word.decode()  # 5 bytes: c, a, f,\xc3\xa9
print(
    f"original word ==> {word} \nencoded word  ==> {encoded_word} \ndecoded word  ==> {decoded_word}"
)

original word ==> café 
encoded word  ==> b'caf\xc3\xa9' 
decoded word  ==> café


#### **Question**: encode an integer value as str or raw?

**INT32 range is :** $(-1 \times 2^{31})$ to $(2^{31}-1)$  
take an example with max int ie :

```python
>>> decimal_value = 2 ** 31 - 1
2147483647
```


In [53]:
decimal_value = 2**31 - 1  # int32 range is -1 * 2 ** 31 to 2 ** 31
num_bytes = (
    4  # int32 ie 4 bytes so  Number of bytes for the desired byte representation = 4
)
bytes_decimal = decimal_value.to_bytes(num_bytes, "big")
print(f"{'decoding raw decimal'.ljust(22)}==> {len(bytes_decimal)} bytes")

bytes_str_decimal = bytes(str(decimal_value), encoding="utf-8")
print(f"{'decoding str-decimal'.ljust(22)}==> {len(bytes_str_decimal)} bytes")

decoding raw decimal  ==> 4 bytes
decoding str-decimal  ==> 10 bytes


### Rules when displaying bytes:

- For bytes with decimal codes 32 to 126—from space to ~ (tilde)—the ASCII char‐
  acter itself is used.
- For bytes corresponding to tab, newline, carriage return, and \, the escape
  sequences \t, \n, \r, and \\ are used.
- If both string delimiters ' and " appear in the byte sequence, the whole sequence
  is delimited by ', and any ' inside are escaped as \'.
  > ```python
  > >>> byte_sequence = b'Hello "world"\'s example'
  > >>> string_representation = byte_sequence.decode('utf-8')
  >
  > >>> print(string_representation)
  >  Hello "world"'s example
  > ```
- For other byte values, a hexadecimal escape sequence is used (e.g., \x00 is the
  null byte)


In [17]:
word = "café"
encoded_word = word.encode("utf8")

print(f"{len(encoded_word)} bytes exist in encoded_word")
print(
    "Based on utf8 ==> every  c, a, f corresponds to one byte and é corresponds to two bytes"
)
# so print every char with its corresponding bytes
# encoded_word[0].to_bytes(1, byteorder='big') equals to encoded_word[0:1]
print(
    f"c ==> {encoded_word[0]:-<9} or {hex(encoded_word[0]):-<9} or {encoded_word[0:1]}\
\na ==> {encoded_word[1]:-<9} or {hex(encoded_word[1]):-<9} or {encoded_word[1:2]}\
\nf ==> {encoded_word[2]:-<9} or {hex(encoded_word[2]):-<9} or {encoded_word[2:3]}\
\né ==> {int.from_bytes(encoded_word[3:], byteorder='big'):-<9} or \
{hex(int.from_bytes(encoded_word[3:], byteorder='big')):-<9} or {encoded_word[3:]}"
)

5 bytes exist in encoded_word
Based on utf8 ==> every  c, a, f corresponds to one byte and é corresponds to two bytes
c ==> 99------- or 0x63----- or b'c'
a ==> 97------- or 0x61----- or b'a'
f ==> 102------ or 0x66----- or b'f'
é ==> 50089---- or 0xc3a9--- or b'\xc3\xa9'


### bytearray vs bytes


| Type      | status    |
| --------- | --------- |
| bytes     | immutable |
| bytearray | mutable   |


In [18]:
print(cafe := bytes("café", encoding="utf_8"))
print(cafe2 := bytearray(cafe))

b'caf\xc3\xa9'
bytearray(b'caf\xc3\xa9')


In [22]:
def mutable_or_not(cafe):
    try:
        cafe[0] = 12
        print(type(cafe), "is mutable")
    except Exception as e:
        print(e)


mutable_or_not(cafe)
mutable_or_not(cafe2)

'bytes' object does not support item assignment
<class 'bytearray'> is mutable


In [32]:
open("test_file", "wb").write(encoded_word)
open("test_file2", "w").write(decoded_word)

4

In [37]:
import keyword
import builtins

keywords = keyword.kwlist
"bytearray" in keywords, "bytearray" in dir(builtins)

(False, True)

#### what is CRLF?

- Windows: Windows-based systems traditionally use the CRLF sequence (CR followed by LF) to represent line endings in text files.
- Unix-like systems (Linux, macOS, etc.): Unix-like systems typically use the LF character alone to represent line endings.


In [23]:
# open("test_file1", 'w').write("hi\nNEWLINE")
open("test_file2", "wb").write(b"Hi\nNEWLINE2")
open("test_file2", "rb").read()

b'Hi\nNEWLINE2'

In [24]:
"Hi\nNEWLINE".encode()

b'Hi\nNEWLINE'

## issues when encodig or decoding


### encoding


In [5]:
city = "São Paulo"
city.encode("cp437")

UnicodeEncodeError: 'charmap' codec can't encode character '\xe3' in position 1: character maps to <undefined>

In [6]:
str.isascii(city)

False

In [8]:
city.encode("cp437", errors="ignore")
# The error='ignore' handler skips characters that cannot be encoded
# this is usually a very bad idea, leading to silent data los

b'So Paulo'

In [139]:
city.encode("cp437", errors="replace")
# When encoding, error='replace' substitutes unencodable characters with '?'
# data is also lost, but users will get a clue that something is amiss.

b'S?o Paulo'

In [140]:
city.encode("cp437", errors="xmlcharrefreplace")
# 'xmlcharrefreplace' replaces unencodable characters with an XML entity.
# If you can’t use UTF, and you can’t afford to lose data, this is the only option.

b'S&#227;o Paulo'

### decoding


many legacy 8-bit encodings like `cp1252`, `iso8859_1`, `koi8_r` **are able to decode any stream of bytes, including random noise, without reporting errors.**


In [143]:
octets = b"Montr\xe9al"
octets.decode("koi8_r")

'MontrИal'

In [144]:
octets.decode("utf_8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 5: invalid continuation byte

In [145]:
octets.decode("utf_8", errors="replace")
# Using 'replace' error handling, the \xe9 is replaced by “�” (code point
# U+FFFD), the official Unicode REPLACEMENT CHARACTER intended to represent
# unknown characters.

'Montr�al'

### When running a module


If you load a .py module containing non-UTF-8 data and no encoding declaration, you get a message like this:

**SyntaxError:**  
Non-UTF-8 code starting with '\xe1' in file ola.py on line1,  
 but no encoding declared; see https://python.org/dev/peps/pep-0263/
for details

a likely scenario is opening a .py file created on Windows with cp1252. Note that this error hap‐
pens even in Python for Windows, because the default encoding for Python 3 source
is UTF-8 across all platforms

solution: at the beginning:

```python
# coding: cp1252
print('Olá, Mundo!')
```


## How to Discover the Encoding of a Byte Sequence


**Short answer: you can’t**

| Sign in Bytes                                          | Conclusion                               | Why                                                                 |
| ------------------------------------------------------ | ---------------------------------------- | ------------------------------------------------------------------- |
| contain byte values over 127                           | not ASCII                                | ascii is 0 to 127                                                   |
| b'\x00' bytes are common                               | 32-bit encoding, and not an 8-bit scheme | null characters in plain text are bugs                              |
| b'\x20\x00' bytes are common                           | UTF-16LE                                 | more likely to represent the space character (U+0020) in a UTF-16LE |
| detect BOM at starting <br/> ex: b'\xef\xbb\xbf\ ... ' | see b'\xef\xbb\xbf\ so likely UTF-8 file | b'\xef\xbb\xbf' BOM of utf-8-sig <br/> other codec may have BOM too |

**note** : A good library to detect encoding is [chardet](https://pypi.org/project/chardet/)


#### Big endian vs little endian


Understanding endianness is important when working with multi-byte values, such as integers, floating-point numbers, and character encodings like UTF-16. It helps ensure that the bytes are interpreted correctly based on the chosen byte order

_least significant byte(LSB)_ is stored at the lowest memory address, while the _most significant byte (MSB)_ is stored at the highest memory address

**Example:**

> **big-endian (storing 0x1234 or 4660):**  
> |memory address : 0x1000 |memory address : 0x1001 |
> |---------|---------|
> |value: 0x12 | value: 0x34 |  
> <br/>

> **little-endian (storing 0x1234 or 4660):**  
> |memory address : 0x1000 |memory address : 0x1001 |
> |---------|---------|
> |value: 0x34 | value: 0x12 |

see cpu standard of your system:

```python
import sys
>>> sys.byteorder
'little'
```


### What is BOM(byte order mark) ?

```python
 u16 = 'El Niño'.encode('utf_16')
>>> u16
b'\xff\xfeE\x00l\x00 \x00N\x00i\x00\xf1\x00o\x00'
```

**\xff\xfe &rarr; BOM**

in utf-16le or utf-16le BOM is not generated

example of little-endian and big-endian in utf-16 encoding:  
e(U+0065) &rarr; 2 Byte : (LSB)&rarr; 0x65 | (MSB)&rarr; 0x00

- utf-16le :

  > Byte 0: 0x65  
  > Byte 1: 0x00 <br/>

- utf-16be :
  > Byte 0: 0x00  
  > Byte 1: 0x65

```python
>>> u16le = 'El Niño'.encode('utf_16le')
>>> list(u16le)
[69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111, 0]
>>> u16be = 'El Niño'.encode('utf_16be')
>>> list(u16be)
[0, 69, 0, 108, 0, 32, 0, 78, 0, 105, 0, 241, 0, 111]
```

**One big advantage of UTF-8 is that it produces the same byte sequence regardless of machine endianness, so no BOM is needed.**

```python
>>> 'El Niño'.encode('utf_8')
b'El Ni\xc3\xb1o'
>>> 'El Niño'.encode('utf_8_sig')
b'\xef\xbb\xbfEl Ni\xc3\xb1o'
```

[codecs documentation](https://docs.python.org/3/library/codecs.html#encodings-and-unicode) says: “In UTF-8, the use of the BOM is discouraged and should generally be avoided.”


## Handling Text Files

recommand specify encoding when write or read txt files


In [26]:
open("cafe.txt", "w", encoding="utf_8").write("café")

4

In [27]:
f = open("cafe.txt", encoding="utf-8")
import os

print(
    f"file content : {f.read()} \nfile size : {os.stat('cafe.txt').st_size}\nfile encoding : {f.encoding}"
)
f.close()

file content : café 
file size : 5
file encoding : utf-8


**Note**: If you omit the encoding argument when opening a file, the default is given by :


In [33]:
import locale

locale.getpreferredencoding()

'UTF-8'

**str <span style="font-size: larger;"> ⇔ </span> locale.getpreferredencoding() <span style="font-size: larger;"> ⇔ </span> bytes**


```bash
chcp
$$ Active code page: 437
```

**Code page 437**, IBM PC-437 or simply **CP437**, was an early character **encoding used by IBM in its original IBM PC** and compatible systems.


In [34]:
!python  utils/default_encodings.py # page 134
# stdout ==> display
# stdin ==> type input
# stderr ==> error showing

 locale.getpreferredencoding() -> 'UTF-8'
                 type(my_file) -> <class '_io.TextIOWrapper'>
              my_file.encoding -> 'UTF-8'
           sys.stdout.isatty() -> True
           sys.stdout.encoding -> 'utf-8'
            sys.stdin.isatty() -> True
            sys.stdin.encoding -> 'utf-8'
           sys.stderr.isatty() -> True
           sys.stderr.encoding -> 'utf-8'
      sys.getdefaultencoding() -> 'utf-8'
   sys.getfilesystemencoding() -> 'utf-8'


It’s weird that chcp and sys.stdout.encoding
say different things when stdout is writing to the console, but it’s great that now we
can print Unicode strings without encoding errors on Windows—unless the user
redirects output to a file That does not mean all your favorite emojis will appear in the console: that also depends on the font the console is using


**Fortunately these codes run well in win11 terminal!** based on fluent python book these codes will not run well on win10  
if see any problem : console font doesn’t have the glyph to display it


In [35]:
import sys
from unicodedata import name

print(sys.version)
print()
print("sys.stdout.isatty():", sys.stdout.isatty())
print("sys.stdout.encoding:", sys.stdout.encoding)
print()
test_chars = [
    "\N{HORIZONTAL ELLIPSIS}",  # exists in cp1252, not in cp437
    "\N{INFINITY}",  # exists in cp437, not in cp1252
    "\N{CIRCLED NUMBER FORTY TWO}",  # not in cp437 or in cp1252
]
for char in test_chars:
    print(f"Trying to output {name(char)}:")
    print(char)

3.10.13 (main, Oct 18 2023, 01:54:08) [GCC 11.3.1 20221121 (Red Hat 11.3.1-4)]

sys.stdout.isatty(): False
sys.stdout.encoding: UTF-8

Trying to output HORIZONTAL ELLIPSIS:
…
Trying to output INFINITY:
∞
Trying to output CIRCLED NUMBER FORTY TWO:
㊷


when python version > 3.6 :

> If PYTHONLEGACYWINDOWSSTDIO is not set or is set to an empty string, the encoding for standard I/O is as follows:
>
> - For interactive I/O (when running Python interactively in a terminal), the encoding is UTF-8.
> - If the output/input is redirected to/from a file, the encoding is determined by locale.getpreferredencoding().


## Normalizing unicode


| unicodedata.normalize(normalization*from*, char\_) | meaning                                                                                     |
| -------------------------------------------------- | ------------------------------------------------------------------------------------------- |
| **NFC**                                            | to produce the shortest equivalent string (safe)                                            |
| NFD                                                | expanding composed characters into base characters and separate combining characters (safe) |
| NFKC                                               | The goal is to have a standardized form for compatibility characters (search&index)         |
| NFKD                                               | The goal is to have a standardized form for compatibility characters (search&index)         |

```python
>>> from unicodedata import normalize
>>> s1 = 'café'
>>> s2 = 'cafe\N{COMBINING ACUTE ACCENT}'
>>> len(s1), len(s2)
(4, 5)
>>> len(normalize('NFC', s1)), len(normalize('NFC', s2))
(4, 4)
>>> len(normalize('NFD', s1)), len(normalize('NFD', s2))
(5, 5)
>>> normalize('NFC', s1) == normalize('NFC', s2)
True
>>> normalize('NFD', s1) == normalize('NFD', s2)
True
```

Keyboard drivers usually generate composed characters, so text typed by users will be
in NFC by default.


**Note:** **NFKC or NFKD may lose or distort information**, but they can produce convenient intermediate representations for searching and indexing


In [17]:
from unicodedata import normalize, name

half = "\N{VULGAR FRACTION ONE HALF}"
print(half, " ==> ", normalize("NFKC", half))

for char in normalize("NFKC", half):
    print(char, name(char), sep="\t")
four_squared = "4²"
print(four_squared, " ==> ", normalize("NFKC", four_squared))

½  ==>  1⁄2
1	DIGIT ONE
⁄	FRACTION SLASH
2	DIGIT TWO
4²  ==>  42


### case folding

case folding is usefull When preparing text for searching or indexing  
**Case folding is essentially converting all text to lowercase, with some additional**
transformations. It is supported by the str.casefold() method  
There are nearly 300 code points for which str.casefold() and str.lower() return
different results.  
It is **good when case-insensitive comparisons but with some problems**  
recommand to use **normalize('NFC', str1).casefold()**

```python
>>> eszett = 'ß'
>>> name(eszett)
'LATIN SMALL LETTER SHARP S'
>>> eszett_cf = eszett.casefold()
>>> eszett, eszett_cf
('ß', 'ss')
```


| purpose                                                     | recommend                         |
| ----------------------------------------------------------- | --------------------------------- |
| safe normalization                                          | NFC normalization                 |
| case-insentive search                                       | NFC + casefold()                  |
| decompose character into base character and combining marks | NFD                               |
| check \_chr\_\_ to find out diacritics?                     | unicodedata.combining(\_char\_\_) |


### sort str

Python sorts sequences of any type by comparing the items in each sequence one by one.  
For strings, this means comparing the code points. Unfortunately, this produces unacceptable results for anyone who uses non-ASCII characters  
**pyuca : pure-Python implementation of the Unicode Collation Algorithm (UCA)**


```python
import locale
my_locale = locale.setlocale(locale.LC_COLLATE, 'pt_BR.UTF-8')
print(my_locale)
fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
sorted_fruits = sorted(fruits, key=locale.strxfrm)
print(sorted_fruits)

```

```python
>>> import pyuca
>>> coll = pyuca.Collator()
>>> fruits = ['caju', 'atemoia', 'cajá', 'açaí', 'acerola']
>>> sorted_fruits = sorted(fruits, key=coll.sort_key)
>>> sorted_fruits
['açaí', 'acerola', 'atemoia', 'cajá', 'caju']
```


## The Unicode Database


you can see more information about characters  
see this char('😸') info from [compart.com](https://www.compart.com/en/unicode/U+1F638)

the Unicode database records whether a character is printable, is a letter, is
a decimal digit, or is some other numeric symbol. That’s how the str methods isal
pha, isprintable, isdecimal, and isnumeric work. str.casefold also uses infor‐
mation from a Unicode table.

**tip:** more information on [unicodedata](https://docs.python.org/3/library/unicodedata.html)


In [47]:
import unicodedata
import re

re_digit = re.compile(r"\d")
# regex is more usefull: https://pypi.org/project/regex/


def better_print(input_expalin, value):
    print(f"{input_expalin:-<35} {value}")


def info_char(character):
    code_point = ord(character)
    unicode_code = f"U+{code_point:04X}"
    better_print(f"The U+ code for {character} ", unicode_code)
    better_print(f"name of {character} char", unicodedata.name(character))
    better_print("char is decimal ?", character.isdecimal())
    better_print(f"char is digit ?", bool(re_digit.match(character)))
    better_print("char is number ? ", character.isnumeric())
    try:  # or you could first check it is a number so see its number to avoid error rasing
        better_print("equal number of char sign ", unicodedata.numeric(character))
    except:
        print("char is not numeric!")


character0 = "\N{CIRCLED NUMBER FORTY TWO}"  # or  "㊷"
character1 = "\N{GRINNING CAT FACE WITH SMILING EYES}"  #  or '😸'
character2 = "\u2466"  # ⑦
character3 = "5"
info_char(character2)

The U+ code for ⑦ ----------------- U+2466
name of ⑦ char--------------------- CIRCLED DIGIT SEVEN
char is decimal ?------------------ False
char is digit ?-------------------- False
char is number ? ------------------ True
equal number of char sign --------- 7.0


## Dual-Mode str and bytes APIs


### re api

regular expressions is valid on str and bytes,  
**in bytes api, bytes outside the ASCII range are treated as nondigits and nonword characters.**


In [83]:
import re

re_numbers_str = re.compile(r"\d+")
re_words_str = re.compile(r"\w+")
re_numbers_bytes = re.compile(rb"\d+")
re_words_bytes = re.compile(rb"\w+")
text_str = "Ramanujan saw \u0be7\u0bed\u0be8\u0bef" " as 1729 = 1³ + 12³ = 9³ + 10³."
text_bytes = text_str.encode("utf_8")
print(f"Text\n {text_str!r}")
print("Numbers")
print(" str :", re_numbers_str.findall(text_str))
print(" bytes:", re_numbers_bytes.findall(text_bytes))
print("Words")
print(" str :", re_words_str.findall(text_str))
print(" bytes:", re_words_bytes.findall(text_bytes))

Text
 'Ramanujan saw ௧௭௨௯ as 1729 = 1³ + 12³ = 9³ + 10³.'
Numbers
 str : ['௧௭௨௯', '1729', '1', '12', '9', '10']
 bytes: [b'1729', b'1', b'12', b'9', b'10']
Words
 str : ['Ramanujan', 'saw', '௧௭௨௯', 'as', '1729', '1³', '12³', '9³', '10³']
 bytes: [b'Ramanujan', b'saw', b'as', b'1729', b'1', b'12', b'9', b'10']


### os api

The GNU/Linux kernel is not Unicode savvy, so in the real world you may find file‐
names made of byte sequences that are not valid in any sensible encoding scheme,
and cannot be decoded to str, If one such function is called with a str
argument, the argument will be automatically converted using the codec named by
sys.getfilesystemencoding(), and the OS response will be decoded with the same
codec.


In [85]:
import os

os.listdir("."), os.listdir(b".")

(['2',
  'cafe.txt',
  'ch4_v1.ipynb',
  'char_name.jpg',
  'dummy',
  'example_encoding.jpg',
  'test_file',
  'test_file2',
  'utils',
  '__pycache__'],
 [b'2',
  b'cafe.txt',
  b'ch4_v1.ipynb',
  b'char_name.jpg',
  b'dummy',
  b'example_encoding.jpg',
  b'test_file',
  b'test_file2',
  b'utils',
  b'__pycache__'])

### Lecturers

1. Mohammad Ansarifard, present date : 04-13-2023, `Linkedin` : [linkedin.com/in/mohammad-ansarifard](https://www.linkedin.com/in/mohammad-ansarifard)
2. Mehrdad biukian date: 04-10-2023, [LinkedIn](www.linkedin.com/in/mehrdad-biukian-naeini)

#### Reviewers

1.
